In [54]:
import pandas as pd

In [55]:
import tensorflow as tf
import pandas as pd
import numpy as np
import ktrain
from ktrain import text
import tensorflow as tf
import seaborn as sns
from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer

In [56]:
tf.__version__

'2.11.0'

In [4]:
df = pd.read_csv('/Users/isingh/Desktop/IMDB_Dataset_cleaned_new.csv')


In [5]:
df.head()

,review,sentiment
0,reviewers mentioned watching oz episode youll ...,positive
1,wonderful production filming technique unassum...,positive
2,wonderful spend time hot summer weekend sittin...,positive
3,basically family boy jake thinks zombie closet...,negative
4,petter matteis love time money visually stunni...,positive


In [6]:
# reviews = df['review'].values
# number of positive and negative reviews
df['sentiment'].value_counts()

positive    25000
negative    25000
Name: sentiment, dtype: int64

In [7]:
TRAIN_SIZE = 40000
TEST_SIZE = 10000

data_train = df[:TRAIN_SIZE]
data_test = df[TRAIN_SIZE:].reset_index(drop=True)

In [8]:
data_train.head()

,review,sentiment
0,reviewers mentioned watching oz episode youll ...,positive
1,wonderful production filming technique unassum...,positive
2,wonderful spend time hot summer weekend sittin...,positive
3,basically family boy jake thinks zombie closet...,negative
4,petter matteis love time money visually stunni...,positive


In [9]:
data_test.head()

,review,sentiment
0,lean liberal political scale found movie offen...,negative
1,excited sitcom represent indian candians found...,negative
2,cover read stuff entirely different type movie...,negative
3,counted appearance dennis hopper make complete...,negative
4,movie tv day enjoy george jungle comedy sequel...,negative


In [10]:
print("Size of train dataset: ",data_train.shape)
print("Size of test dataset: ",data_test.shape)

Size of train dataset:  (40000, 2)
Size of test dataset:  (10000, 2)


In [11]:
(X_train, y_train), (X_test, y_test), preproc = text.texts_from_df(train_df=data_train,text_column = 'review',label_columns = 'sentiment',val_df = data_test,maxlen = 500,ngram_range=2,preprocess_mode = 'bert')

['negative', 'positive']
   negative  positive
0       0.0       1.0
1       0.0       1.0
2       0.0       1.0
3       1.0       0.0
4       0.0       1.0
['negative', 'positive']
   negative  positive
0       1.0       0.0
1       1.0       0.0
2       1.0       0.0
3       1.0       0.0
4       1.0       0.0
preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [12]:
len(X_train[1])

40000

In [13]:
X_train[0].shape

(40000, 500)

In [14]:
model = text.text_classifier(name = 'bert',train_data = (X_train, y_train),preproc = preproc)

Is Multi-Label? False
maxlen is 500


/Users/isingh/opt/miniconda3/lib/python3.9/site-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer GlorotNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


done.


In [15]:
# fit the mode
learner = ktrain.get_learner(model = model,train_data = (X_train, y_train),val_data = (X_test, y_test),batch_size = 6)

In [45]:
import transformers
# tokenizer import 
#from transformers import BertTokenizer


In [46]:
transformer = transformers.TFDistilBertModel.from_pretrained('distilbert-base-uncased')

In [35]:
MAX_VOCAB_SIZE = 10000

In [50]:
tokenizer = Tokenizer(num_words = MAX_VOCAB_SIZE,oov_token="<oov>")

In [42]:
# create a sequence of reviews
reviews = df['review'].values


In [48]:
seq_train = tokenizer.texts_to_sequences([X_train])
seq_test =  tokenizer.texts_to_sequences([X_test])

AttributeError: 'DistilBertTokenizer' object has no attribute 'texts_to_sequences'

In [105]:
# fit the model
learner.fit_onecycle(lr = 2e-5, epochs = 1)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/3
 119/6667 [..............................] - ETA: 148:29:45 - loss: 0.6983 - accuracy: 0.5210

In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc)
predictor.save("/Users/isingh/Desktop/IMDB_Dataset_cleaned_new.csv")